# 介绍
要通过Tensorflow进行机器学习，你可能需要定义、保存和恢复模型。

一个模型，抽象的说,它是：
 - 一个计算张量的函数(前向传递)
 - 能通过训练而更新的一些变量
 
在本指南中，您将深入Keras的底层，了解TensorFlow模型是如何定义的。同时，将介绍TensorFlow如何收集变量和模型，以及如何保存和恢复它们。

# 模块引入

In [ ]:
import tensorflow as tf
from datetime import datetime

%load_ext tensorboard

# 通过Tensorflow定义模型和层
大多数模型是由层组成的。层是具有已知数据结构的函数，可以重用，并具有可训练的变量。在TensorFlow中，层和模型的大多数高级实现(如Keras或Sonnet)都构建在相同的基础类tf.Module上。

以下是一个操作标量张量的tf.Moduled的简单例子：

In [ ]:
class SimpleModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.a_variable = tf.Variable(5.0, name="train_me")
    self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
    
  def __call__(self, x):
    return self.a_variable * x + self.non_trainable_variable

simple_module = SimpleModule(name="simple")
simple_module(tf.constant(5.0))

Modules，甚至layers，都可以理解为深度学习术语的”对象“：都有各自的内部状态，以及使用该状态的方法。

call()没有什么特殊性，只是它看起来是一个python中可调用的函数；你可以使用任何你所希望的函数来调用你的模型。

您可以依据任何原因设置变量的可训练性，包括在微调期间冻结(freeze)层和变量。

> tf.Module是tf.keras.layers.Layer和tf.keras.Model的基类，所以你在这里看到的一切也适用于Keras。由于历史兼容性的原因，Keras层不从模块中收集变量，所以您的模型应该只使用模块或仅使用Keras层。但是，下面显示的检查变量的方法在这两种情况下都是相同的。

通过子类化tf.Module，分配这个对象属性的任何tf.Variable或tf.Module实例都将自动收集。这将允许你保存和加载变量，同时也创建tf.Module的集合。

In [ ]:
# 所有训练变量
print("trainable variables:", simple_module.trainable_variables)
# 所有变量
print("all variables:", simple_module.variables)

这是一个由模块组成的两层线性层模型的例子。 首先是一个线性(Dense)层：

In [ ]:
class Dense(tf.Module):
  def __init__(self, in_features, out_features, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
    
  def __call__(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

然后是一个创建并应用了两个层实例的完整模型：

In [ ]:
class SequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# 你已经构建了一个模型
my_model = SequentialModule(name="the_model")
# 通过随机值来调用模型
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

tf.Module实例将自动收集分配给它的任何tf.Variable或tf.Module实例。这允许您使用单个模型实例管理tf.Module的一个集合，并保存和加载整个模型。

In [ ]:
print("Submodules:", my_model.submodules)

In [ ]:
for var in my_model.variables:
  print(var, "\n")

## 等待创建变量
您可能已经注意到，您必须定义该层的输入和输出sizes。这是为了使变量w有一个已知的shape并且可以被分配。

通过将变量创建延迟到第一次使用特定输入shape调用模块时，您不需要预先指定输入大小。

In [ ]:
class FlexibleDenseModule(tf.Module):
  # 无需”in+features“
  def __init__(self, out_features, name=None):
    super().__init__(name=name)
    self.is_built = False
    self.out_features = out_features

  def __call__(self, x):
    # 在第一次调用时创建变量
    if not self.is_built:
      self.w = tf.Variable(
        tf.random.normal([x.shape[-1], self.out_features]), name='w')
      self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
      self.is_built = True

    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

# 在模块中使用
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = FlexibleDenseModule(out_features=3)
    self.dense_2 = FlexibleDenseModule(out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

my_model = MySequentialModule(name="the_model")
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

这种灵活性就是为什么TensorFlow层通常只需要指定其输出的shape(比如tf.keras.layers.Dense)，而不需要同时指定输入和输出的size。

# 保存权重
你可以将tf.Module同时保存为一个检查点(checkpoint)和SavedModel。 检查点也就是权重(即模块及其子模块的内部的一组变量集的值)。

In [ ]:
chkp_path = "my_checkpoint"
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)
checkpoint.write(chkp_path)

检查点由两种文件组成——数据本身，以及元数据的索引文件。索引文件记录实际保存的内容和检查点的编号，而检查点数据包含变量值及其属性查找路径。

```bash
ls my_checkpoint*
```

my_checkpoint.data-00000-of-00001  my_checkpoint.index

您可以查看检查点内部，以确保所有变量都已保存，并按包含它们的Python对象排序。

In [ ]:
tf.train.list_variables(chkp_path)

在分布式(多机环境)训练期间，可以对它们进行分片，这就是它们被编号的原因(例如，“00000- 00001”)。但是，在本例中，只有一个分片。

当您重新加载模型时，您将覆盖Python对象中的值。

In [ ]:
new_model = MySequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("my_checkpoint")

# 结果应该和上面一样吗
new_model(tf.constant([[2.0, 2.0, 2.0]]))

由于检查点是长期训练工作流程的核心，tf.checkpoint.CheckpointManager是一个使检查点管理更加容易的工具类。

# 保存函数
TensorFlow可以在没有原始Python对象的情况下运行模型，这在TensorFlow Serving和TensorFlow Lite中可以看到，甚至当TensorFlow Hub下载训练过的模型时也可以看到。

TensorFlow需要知道如何执行Python中描述的计算，但无需原始代码。为此，您可以制作一个graph，这在前面的指南中有描述。

这个graph包含了实现这个函数的操作。

您可以在上面的模型中定义一个graph，通过添加@tf.function装饰器来指示该代码应该作为graph运行。

In [ ]:
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  @tf.function
  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# 你已经通过graph构建了一个model
my_model = MySequentialModule(name="the_model")

您所创建的模块与以前完全相同。传入函数的每个唯一签名都创建一个单独的graph。

In [ ]:
print(my_model([[2.0, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

您可以通过在TensorBoard summary中跟踪图graph来可视化该graph。

In [ ]:
# 设置日志记录
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

# 创建新模型以获得新的跟踪
# 否则，summary将不会看到graph。
new_model = MySequentialModule()

# 用括号括起函数调用
# tf.summary.trace_on() 和tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)

# Call only one tf.function when tracing.
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

通过tensorboard查看结果轨迹

In [ ]:
%tensorboard --logdir logs/func

## 创建一个SavedModel
共享完整训练模型的推荐方式是使用SavedModel。SavedModel同时包含函数集合和权重集合。

您可以保存刚刚创建的模型。

In [ ]:
tf.saved_model.save(my_model, "the_saved_model")

```bash
# 查看保存目录内容
!ls -l the_saved_model
```

total 24
drwxr-sr-x 2 kbuilder kokoro  4096 Sep 10 01:35 assets
-rw-rw-r-- 1 kbuilder kokoro 12617 Sep 10 01:35 saved_model.pb
drwxr-sr-x 2 kbuilder kokoro  4096 Sep 10 01:35 variables

```bash
# 目录variables包含变量的检查点
ls -l the_saved_model/variables
```

total 8
-rw-rw-r-- 1 kbuilder kokoro 408 Sep 10 01:35 variables.data-00000-of-00001
-rw-rw-r-- 1 kbuilder kokoro 356 Sep 10 01:35 variables.index

saved_model.pb文档是一个描述函数tf.Graph的 protocol buffer协议缓冲区。

模型和层可以从这个表示中加载，而不需要实际创建它的类的实例。这在没有(或不需要)Python解释器的情况下是需要的，例如在大规模或在边缘设备上服务，或者在原始Python代码不可用或无法实际使用的情况下。

您可以将模型作为新对象加载：

In [ ]:
new_model = tf.saved_model.load("the_saved_model")

new_model创建于一个加载saved model，它是一个内部的Tensorflow用户对象。它与SequentialModule不是同一类型。 

In [ ]:
isinstance(new_model, SequentialModule)

这个新模型在已经定义的输入签名上工作。您不能向这样恢复的模型添加更多签名。

In [ ]:
print(my_model([[2.0, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

因此，使用SavedModel，您就能够使用tf.Module保存Tensorflow权重和计算图，然后再次加载它们。

# Keras模型和层
请注意，在此之前，还没有提到Keras。您可以在tf.Module之上构建自己的高级API，且人们已经这样做了。

在本节中，您将研究Keras如何使用tf.Module。Keras模型的完整用户指南可以在Keras指南中找到。

## Keras layers
tf.keras.layers.Layer是所有Keras层的基类，它继承自tf.Module。

你只需要换出其父模块和将__call__改为call就可以将一个模块转换成一个Keras层：

In [ ]:
class MyDense(tf.keras.layers.Layer):
  # 添加kwargs支持基础Keras层参数
  def __init__(self, in_features, out_features, **kwargs):
    super().__init__(**kwargs)

    # 这将移到build步，参见后面
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
    
  def call(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

simple_layer = MyDense(name="simple", in_features=3, out_features=3)

Keras层有自己的__call__，它做一些在下一节中描述的记账工作，然后调用call()。您应该不会看到功能上的变化。

In [ ]:
 simple_layer([[2.0, 2.0, 2.0]])

## 构建步骤
如前所述，在许多情况下，等到确定了输入shape后再创建变量是很方便的。

Keras层带有一个额外的生命周期步骤，允许您在如何定义您的层方面有更多的灵活性。这在build()函数中定义。

build只调用一次，并且调用时使用的是输入的shape。它通常用于创建变量(权重)。

你可以灵活地根据输入的size来重写上面的MyDense层。

In [ ]:
class FlexibleDense(tf.keras.layers.Layer):
  # 添加kwargs`, Keras支持许多参数
  def __init__(self, out_features, **kwargs):
    super().__init__(**kwargs)
    self.out_features = out_features

  def build(self, input_shape):  # 创建层的状态
    self.w = tf.Variable(
      tf.random.normal([input_shape[-1], self.out_features]), name='w')
    self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

  def call(self, inputs):  # 定义从输入到输出的计算
    return tf.matmul(inputs, self.w) + self.b

# 创建层的实例
flexible_dense = FlexibleDense(out_features=3)

此时，模型还没有建立，因此没有变量。

In [ ]:
flexible_dense.variables

调用该函数可以分配适当大小的变量。

In [ ]:
# 调用，产生随机结果
print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0], [3.0, 3.0, 3.0]])))

In [ ]:
flexible_dense.variables

由于build只被调用一次，如果输入shape与层的变量不兼容，输入将被拒绝。

In [ ]:
try:
  print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0, 2.0]])))
except tf.errors.InvalidArgumentError as e:
  print("Failed:", e)

Keras层有很多额外的特征，包括:
- 可选的损失
- 对指标的支持
- 内置对可选训练参数的支持，以区分训练和推理的使用
- get_config和from_config方法，允许您准确地存储配置，以允许在Python中克隆模型

请在[自定义层](https://tensorflow.google.cn/guide/keras/custom_layers_and_models)的完整指南中了解它们。

## Keras模型
您可以将您的模型定义为嵌套的Keras层。

然而，Keras还提供了一个名为tf.keras.Model的全功能模型类。它继承于tf.keras.layers.Layer，所以一个Keras模型也是Keras层，可以以同样的方式使用、嵌套和保存。Keras模型带有额外的功能，使它们易于训练、评估、加载、保存，甚至在多台机器上训练。

您可以从上面用几乎相同的代码定义SequentialModule，再一次将剩余的__call__转换为call()并更改父模块。

In [ ]:
class MySequentialModel(tf.keras.Model):
  def __init__(self, name=None, **kwargs):
    super().__init__(**kwargs)

    self.dense_1 = FlexibleDense(out_features=3)
    self.dense_2 = FlexibleDense(out_features=2)
    
  def call(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# 你拥有一个Keras模型
my_sequential_model = MySequentialModel(name="the_model")

# 通过张量调用，产生随机结果
print("Model results:", my_sequential_model(tf.constant([[2.0, 2.0, 2.0]])))

所有相同的特性都是可用的，包括跟踪变量和子模块。

In [ ]:
my_sequential_model.variables

In [ ]:
my_sequential_model.submodules

(<__main__.FlexibleDense at 0x7fbffc7c2cf8>,
 <__main__.FlexibleDense at 0x7fbffc7c2f98>)

重写tf.keras.Model是构建TensorFlow模型的一种非常python化的方法。如果您正在从其他框架迁移模型，这可能非常简单。

如果您正在构建的模型是现有层和输入的简单组合，那么通过使用函数式API可以节省时间和空间，该API提供了关于模型重构和体系结构的附加特性。

下面是与函数式API相同的模型:

In [ ]:
inputs = tf.keras.Input(shape=[3,])

x = FlexibleDense(3)(inputs)
x = FlexibleDense(2)(x)

my_functional_model = tf.keras.Model(inputs=inputs, outputs=x)
my_functional_model.summary()

In [ ]:
my_functional_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 2.9168186, -3.7479482]], dtype=float32)>

这里的主要区别是，输入shape是预先指定的，作为函数式构建过程的一部分。在这种情况下，不需要完全指定input_shape参数；您可以将某些维度保留为None。

# 保存Keras模型
Keras模型可以有检查点，它看起来和tf.Module一样。

Keras模型也可以通过tf.saved_models.save()保存，因为它们是模块。然而，Keras模型有方便的方法和其他功能。

In [ ]:
my_sequential_model.save("exname_of_file")

同样地，它们也可以被加载回去。

In [ ]:
reconstructed_model = tf.keras.models.load_model("exname_of_file")

Keras SavedModels还可以保存指标、丢失和优化器状态。

当对相同的数据进行调用时，可以使用这个重新构建的模型并将产生相同的结果。

In [ ]:
reconstructed_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-1.1975694, -6.3796988]], dtype=float32)>

关于Keras模型的保存和序列化，还需要了解更多，包括为特性支持提供定制层的配置方法。